In [3]:
pip install pygame

  Using cached pygame-2.6.1-cp311-cp311-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp311-cp311-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Lenevo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
import pygame
import random
import time
import sys

# Initialize Pygame and Mixer
pygame.init()
pygame.mixer.init()

# Colors
WHITE, BLACK, GRAY = (255, 255, 255), (0, 0, 0), (40, 40, 40)
BLUE, RED, YELLOW = (0, 100, 255), (200, 0, 0), (255, 255, 0)
GREEN, FUEL_BLUE, CYAN = (0, 255, 0), (0, 200, 255), (0, 255, 255)
DARK_BLUE = (0, 0, 150)

# Screen Setup
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Nitro Racer: High Speed Edition")
clock = pygame.time.Clock()

# Fonts
font_s = pygame.font.SysFont("Arial", 22, bold=True)
font_m = pygame.font.SysFont("Arial", 30, bold=True)
font_l = pygame.font.SysFont("Arial", 65, bold=True)

ENEMY_COLORS = [(200, 0, 0), (0, 180, 0), (220, 140, 0), (180, 0, 180)]
high_score = 0

# Music Setup
try:
    pygame.mixer.music.load("music.mp3")
    pygame.mixer.music.set_volume(0.4)
except:
    pass

def draw_realistic_car(x, y, main_color, nitro=False, shielded=False):
    w, h = 52, 85
    
    # Nitro Flame Effect
    if nitro:
        for i in range(3):
            pygame.draw.ellipse(screen, YELLOW, [x + 12, y + h + (i*5), 28, 40])
            pygame.draw.ellipse(screen, RED, [x + 16, y + h + (i*8), 20, 30])

    # Shield Glow
    if shielded:
        pygame.draw.ellipse(screen, CYAN, [x - 10, y - 10, w + 20, h + 20], 3)

    # 1. Main Chassis (The body)
    pygame.draw.rect(screen, main_color, [x, y + 10, w, h - 20], border_radius=12)
    
    # 2. The Hood (Front)
    pygame.draw.rect(screen, main_color, [x + 5, y, w - 10, 20], border_top_left_radius=15, border_top_right_radius=15)
    
    # 3. The Roof & Windshield
    pygame.draw.rect(screen, (30, 30, 30), [x + 8, y + 25, w - 16, 25], border_radius=5) # Windshield
    pygame.draw.rect(screen, main_color, [x + 10, y + 32, w - 20, 15]) # Roof
    
    # 4. Spoiler (Back)
    pygame.draw.rect(screen, (20, 20, 20), [x, y + h - 5, w, 8], border_radius=2)
    
    # 5. Headlights
    pygame.draw.rect(screen, WHITE, [x + 8, y + 2, 10, 5], border_radius=2)
    pygame.draw.rect(screen, WHITE, [x + w - 18, y + 2, 10, 5], border_radius=2)

    # 6. Tires
    for ty in [15, 60]:
        pygame.draw.rect(screen, BLACK, [x - 4, y + ty, 6, 15], border_radius=2) # Left
        pygame.draw.rect(screen, BLACK, [x + w - 2, y + ty, 6, 15], border_radius=2) # Right

def game_loop():
    global high_score
    x, y = (WIDTH // 2 - 26), (HEIGHT * 0.75)
    x_change = 0
    enemies, items = [], []
    base_speed, score, fuel = 5, 0, 100.0
    has_shield = False
    
    # ROAD STRIPES (This creates the "Moving Forward" feeling)
    road_stripes = [[WIDTH // 2 - 5, y] for y in range(-100, HEIGHT, 100)]
    
    if not pygame.mixer.music.get_busy():
        try: pygame.mixer.music.play(-1)
        except: pass

    while True:
        keys = pygame.key.get_pressed()
        nitro = keys[pygame.K_SPACE] and fuel > 0
        mult = 2.5 if nitro else 1.0
        fuel = max(0, fuel - 0.7) if nitro else min(100, fuel + 0.15)

        for event in pygame.event.get():
            if event.type == pygame.QUIT: pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT: x_change = -7
                if event.key == pygame.K_RIGHT: x_change = 7
            if event.type == pygame.KEYUP:
                if event.key in [pygame.K_LEFT, pygame.K_RIGHT]: x_change = 0

        x += x_change
        screen.fill(GRAY)

        # MOVE AND DRAW ROAD STRIPES
        for stripe in road_stripes:
            stripe[1] += base_speed * mult # Move stripes down
            pygame.draw.rect(screen, WHITE, [stripe[0], stripe[1], 10, 50])
            if stripe[1] > HEIGHT: stripe[1] = -50

        # SPAWN LOGIC
        if random.randint(1, 40) == 1 and len(enemies) < 4:
            enemies.append([random.randrange(20, WIDTH-70), -120, random.uniform(base_speed, base_speed+2), random.choice(ENEMY_COLORS)])

        if random.randint(1, 500) == 1:
            items.append([random.randrange(50, WIDTH-50), -50, random.choice(['R', 'S'])])

        # UPDATE ENEMIES
        for e in enemies[:]:
            e[1] += e[2] * mult
            draw_realistic_car(e[0], e[1], e[3])
            
            # Collision
            if y < e[1] + 75 and y + 75 > e[1] and x < e[0] + 45 and x + 45 > e[0]:
                if has_shield:
                    has_shield = False; enemies.remove(e)
                else: return score

            if e[1] > HEIGHT:
                enemies.remove(e); score += 1; base_speed += 0.03

        # UPDATE ITEMS
        for i in items[:]:
            i[1] += base_speed * mult
            if i[2] == 'S':
                pygame.draw.circle(screen, YELLOW, (i[0], int(i[1])), 18)
                pygame.draw.circle(screen, CYAN, (i[0], int(i[1])), 18, 2)
            else:
                pygame.draw.rect(screen, GREEN, [i[0]-15, i[1]-15, 30, 30], border_radius=5)
            
            if y < i[1]+20 and y+80 > i[1]-20 and x < i[0]+20 and x+50 > i[0]-20:
                if i[2] == 'S': has_shield = True
                else: fuel = 100
                items.remove(i)
            elif i[1] > HEIGHT: items.remove(i)

        # DRAW PLAYER
        draw_realistic_car(x, y, BLUE, nitro, has_shield)
        
        # DASHBOARD
        pygame.draw.rect(screen, BLACK, [0, HEIGHT-60, WIDTH, 60])
        pygame.draw.rect(screen, (50, 50, 50), [20, HEIGHT-35, 104, 15])
        pygame.draw.rect(screen, FUEL_BLUE, [22, HEIGHT-33, int(fuel), 11])
        screen.blit(font_s.render("FUEL", True, WHITE), (20, HEIGHT-55))
        screen.blit(font_m.render(f"{int(base_speed*12*mult)} KM/H", True, GREEN if not nitro else RED), (WIDTH-140, HEIGHT-45))
        screen.blit(font_s.render(f"SCORE: {score}", True, WHITE), (10, 10))

        if x < 0 or x > WIDTH-52: return score
        pygame.display.update(); clock.tick(60)

# LOSE SCREEN
final_score = game_loop()
high_score = max(high_score, final_score)
screen.fill(BLACK)
display_text("WASTED", font_l, RED, -20)
display_text(f"Final Score: {final_score}", font_s, WHITE, 50)
pygame.display.update()
time.sleep(3); pygame.quit(); sys.exit()

SystemExit: 